# GFW Data-API Playground

When developing, to test newly saved code:

- restart kernel
- after each save in the command line use: `!python setup.py develop ` or `!pip install -e .` to install
- then re-import `LMIPy`
- use `py.test` in cli to execute tests

### To release

1. Increment the version in `setup.py`.
2. Push changes to github
3. Checkout to master, pull
3. Remove the `dist` folder
3. In the CLI run `make pypi`
4. Check version on [Colab](https://colab.research.google.com/drive/1tRye2QAytWzuki3DQyg4G7fsFFKlZNic#scrollTo=EN4iMVRm6N0Y)



In [1]:
!python setup.py develop 

from IPython.display import clear_output,display
clear_output()

import LMIPy as lmi
from LMIPy import utils

print(f'LMI ver. {lmi.__version__} ready!')

/Users/vizzuality/anaconda3/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


LMI ver. 0.5.0 ready!


# RW Auth Class

In [3]:
SERVER = 'production'
# SERVER = 'staging'

creds = lmi.Auth(server=SERVER)

## Register new user

In [4]:
email = 'aj.pain89@gmail.com'
name = 'AJ Pain'

creds.register(email=email, name=name)

## Login and generate token

Note that generating a token will invalidate existing tokens.

In [8]:
email = 'aj.pain89@gmail.com'
creds.login(email=email)

In [10]:
creds.generateToken()

In [9]:
TOKEN = creds.token

In [11]:
creds.role

'USER'

In [12]:
creds.apps

[]

## Update user

In [4]:
email = 'adam.pain@vizzuality.com'
creds.login(email=email)

Password: ········


In [5]:
print(f"user_id: {creds._id}")
print(f"role: {creds.role}")
print(f"apps: {creds.apps}")

TOKEN = creds.token

user_id: 59db4eace9c1380001d6e4c3
role: ADMIN
apps: ['rw', 'gfw', 'prep', 'aqueduct', 'forest-atlas', 'data4sdgs', 'gfw-climate', 'gfw-pro', 'ghg-gdp', 'ng']


In [17]:
user = creds.getUserByEmail(user_email='aj.pain89@gmail.com', token=TOKEN)
user

https://api.resourcewatch.org/auth/user?app=all&email=aj.pain89@gmail.com


[{'_id': '60eda01babdcd1001b768594',
  'createdAt': '2021-07-13T14:15:55.000Z',
  'email': 'aj.pain89@gmail.com',
  'extraUserData': {'apps': []},
  'id': '60eda01babdcd1001b768594',
  'name': 'AJ Pain',
  'provider': 'local',
  'role': 'USER',
  'updatedAt': '2021-07-13T14:17:35.000Z'}]

In [19]:
uid = user[0]["_id"]
uid

'60eda01babdcd1001b768594'

In [22]:
payload = {
    'extraUserData': {
        'apps': ['gfw']
    },
    'role': 'ADMIN'
}
creds.updateUser(user_id=uid, token=TOKEN, payload=payload)

https://api.resourcewatch.org/auth/user/60eda01babdcd1001b768594


{'_id': '60eda01babdcd1001b768594',
 'createdAt': '2021-07-13T14:15:55.000Z',
 'email': 'aj.pain89@gmail.com',
 'extraUserData': {'apps': ['gfw']},
 'id': '60eda01babdcd1001b768594',
 'name': 'AJ Pain',
 'provider': 'local',
 'role': 'ADMIN',
 'updatedAt': '2021-07-13T14:39:49.000Z'}

# Data API

### Get Data-API Keys

In [6]:
from LMIPy import dataAPI

# SERVER = 'production'
SERVER = 'staging'

In [7]:
## Get data-api creds (staging only; not currently enforced in production)
## Always use Prod RW API TOKEN
creds = dataAPI.Auth(server=SERVER, rw_api_token=TOKEN)

In [8]:
## See keys that I have generated
creds.keys

[GFW Data API Credential: 'GFW Flagship Staging Key' (does not expire),
 GFW Data API Credential: 'AJ data-api' (does not expire)]

In [9]:
## Set data-api key
DATA_API_KEY = creds.keys[1].api_key

### Search Data Catalogue

In [10]:
## Search the catalogue to populate datasets
col = dataAPI.DataCatalogue(server=SERVER, search='umd', token=DATA_API_KEY)
datasets = col.datasets
datasets

{1: GFW Data API Dataset: 'umd_glad_alerts__adm1_weekly',
 2: GFW Data API Dataset: 'umd_glad_alerts__adm2_daily',
 3: GFW Data API Dataset: 'umd_glad_landsat_alerts',
 4: GFW Data API Dataset: 'umd_glad_sentinel2_alerts',
 5: GFW Data API Dataset: 'umd_modis_burned_areas',
 6: GFW Data API Dataset: 'umd_regional_primary_forest_2001',
 7: GFW Data API Dataset: 'umd_soy_on_tree_cover_loss',
 8: GFW Data API Dataset: 'umd_soy_planted_area',
 9: GFW Data API Dataset: 'umd_tree_cover_density_2000',
 10: GFW Data API Dataset: 'umd_tree_cover_density_2010',
 11: GFW Data API Dataset: 'umd_tree_cover_gain',
 12: GFW Data API Dataset: 'umd_tree_cover_loss',
 13: GFW Data API Dataset: 'umd_tree_cover_loss__adm2_change'}

In [15]:
## Select Dataset and populate with metadata as needed
d = datasets[5]
d

GFW Data API Dataset: 'umd_modis_burned_areas'

In [16]:
d.get_versions(True)
d.versions

https://staging-data-api.globalforestwatch.org/dataset/umd_modis_burned_areas


['v2020',
 'v2020.3',
 'v2021.0',
 'v2021.2',
 'v2021.3',
 'v2021.4',
 'v2021.7',
 'v2021.8']

In [19]:
## Set a specific version (if no latest)
d.version = d.versions[-1]

In [20]:
## Populate metadata etc
d.get_metadata(verbose=True)
d.get_fields(True)

d.metadata['function']

https://staging-data-api.globalforestwatch.org/dataset/umd_modis_burned_areas/v2021.8
https://staging-data-api.globalforestwatch.org/dataset/umd_modis_burned_areas/v2021.8/fields


In [21]:
d.metadata

{'added_date': None,
 'analysis': None,
 'cautions': None,
 'citation': None,
 'content_date': None,
 'content_date_range': {'max': '2020-12-31', 'min': '2000-09-01'},
 'data_language': None,
 'data_updates': None,
 'download': None,
 'function': None,
 'geographic_coverage': None,
 'key_restrictions': None,
 'last_update': None,
 'learn_more': None,
 'license': None,
 'other': None,
 'overview': None,
 'resolution': None,
 'scale': None,
 'source': None,
 'subtitle': None,
 'tags': None,
 'title': None,
 'update_frequency': None,
 'version_number': None,
 'why_added': None}

### Use Slug Directly

In [22]:
## Instantiate new dataset object using the table name slug directly
dataset_name = 'umd_modis_burned_areas'

d = dataAPI.Dataset(server=SERVER, slug=dataset_name)

In [23]:
## Populate metadata etc
d.get_versions(True)
d.versions

https://staging-data-api.globalforestwatch.org/dataset/umd_modis_burned_areas


['v2020',
 'v2020.3',
 'v2021.0',
 'v2021.2',
 'v2021.3',
 'v2021.4',
 'v2021.7',
 'v2021.8']

In [24]:
d.version = d.versions[-1]

In [30]:
d.get_metadata()
d.get_fields()

# Queryable fields
d.fields

['gfw_fid',
 'alert__date',
 'geom',
 'geom_wm',
 'gfw_area__ha',
 'gfw_geostore_id',
 'gfw_geojson',
 'gfw_bbox',
 'created_on',
 'updated_on']

In [31]:
## Query using a geostore to do zonal stats
gid = 'c3833748f6815d31bad47d47f147c0f0'

In [33]:
sql = """SELECT SUM(gfw_area__ha),
              alert__date
        FROM data
        WHERE alert__date > '2015-01-01'
        GROUP BY alert__date
       """

result = d.query(sql=sql, geostore_id=gid, verbose=True)
result

https://staging-data-api.globalforestwatch.org/dataset/umd_modis_burned_areas/v2021.8/query/json?sql=SELECT+SUM%28gfw_area__ha%29%2C%0A++++++++++++++alert__date%0A++++++++FROM+data%0A++++++++WHERE+alert__date+%3E+%272015-01-01%27%0A++++++++GROUP+BY+alert__date%0A+++++++&geostore_id=c3833748f6815d31bad47d47f147c0f0&geostore_origin=rw


[{'alert__date': '2015-01-24', 'sum': 23.7707400615323},
 {'alert__date': '2015-01-27', 'sum': 23.7707292647075},
 {'alert__date': '2015-01-28', 'sum': 23.7707072627768},
 {'alert__date': '2015-01-30', 'sum': 71.3121386526033},
 {'alert__date': '2015-02-03', 'sum': 71.3121768612035},
 {'alert__date': '2015-02-05', 'sum': 118.8535692487325},
 {'alert__date': '2015-02-06', 'sum': 23.7709871186549},
 {'alert__date': '2015-02-09', 'sum': 47.5417837046964},
 {'alert__date': '2015-02-10', 'sum': 142.6255246053349},
 {'alert__date': '2015-02-11', 'sum': 47.5416984642444},
 {'alert__date': '2015-02-12', 'sum': 95.0835903891303},
 {'alert__date': '2015-02-13', 'sum': 95.0838307072704},
 {'alert__date': '2015-02-14', 'sum': 166.3954259257716},
 {'alert__date': '2015-02-16', 'sum': 118.8543363479301},
 {'alert__date': '2015-02-18', 'sum': 71.3127416537035},
 {'alert__date': '2015-02-19', 'sum': 166.396009346442},
 {'alert__date': '2015-02-20', 'sum': 71.3125024899993},
 {'alert__date': '2015-02-2